In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np 
import random
import os
import io # ? 


C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
filename = "Dost_notepad.txt"
text = open(filename, 'r', encoding='utf-8').read()
text = text.lower()
chars = sorted(list(set(text)))
print("Символьная длина текста:", len(text))
print("Всего символов:", len(chars))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i,c) for i, c in enumerate(chars))

length = 50 # Длина последовательности символов
step = 1
sentences = []
next_chars = []

for i in range(0, len(text) - length, step): 
    sentences.append(text[i : i + length])
    next_chars.append(text[i+length])
    
print('Число последовательностей символов:', len(sentences))

#One-Hot Encoding
x = np.zeros((len(sentences), length, len(chars)), dtype = np.bool) 
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, fragment in enumerate(sentences):
    for j, char in enumerate(fragment):
        x[i, j, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

Символьная длина текста: 233991
Всего символов: 68
Число последовательностей символов: 233941


In [3]:
#models 

def lstm_model():
    model = keras.Sequential()
    model.add(keras.Input(shape = (length, len(chars))))
    model.add(layers.LSTM(16))
    model.add(layers.Dense(len(chars), activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
    
              

In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
model_1 = lstm_model()

weights_path = 'C:\\Users\\Pankratz\\Desktop\\Sequence_generation\\weights'
epochs = 10
batch_size = 256
#checkpoint = ModelCheckpoint(weights_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]

for epoch in range(epochs):
    model_1.fit(x, y, epochs=1, batch_size=batch_size)
    start = random.randint(0, len(text) - length - 1)
    model_output = ""
    fragment = text[start : start + length]
    print('Генерируем начиная с отрывка: "' + fragment + '"')

    diversity = 1.0
    for i in range(400):
        x_pred = np.zeros((1, length, len(chars)))
        for j, char in enumerate(fragment):
            x_pred[0, j, char_to_int[char]] = 1.0
            prediction = model_1.predict(x_pred, verbose = 0)[0]
            index = sample(prediction, diversity)
            next_char = int_to_char[index]
            fragment = fragment[1:] + next_char
            model_output += next_char
        
    print('Сгенерированный текст:', model_output)
    print()


233941/233941 [==============================] - 37s 160us/sample - loss: 3.0964
Генерируем начиная с отрывка: "нь,  сырость;  что  ты,  лежа-то  одна,  тогда  пе"
Сгенерированный текст: ,омиу - оте твя "а нуаеткем р8отчпсош  мьыдоое!, - ьриояиео  увдодрб аиыс
ооолтои-бис а
ховиаоп ылл вюгоаротец,о  н
иулх но  с,я,  сугйо гд ,исисум д иио одп мй-сс оду? иих оацдивевос 
,  иу тс  оао м ут ихь  оруьт ласольт
 врооо!о ,еиь ,о ьиь яитмйбегорт,чг гооееу кре о неихго дь,т, ,ое жща,цеаотиотознртсимчртглуьеен   пк ебяызктшиио й  рме сги еблрч*а иее о
яв л е к оо, .т
ело,йпвевивтдтч ь
омлдредотеогопву я лта отися,в  е  ао явьады   ,иеньъс анчлолорутод)т  еномя ичoожgаз  рьяаоочаичсиоо  во йге.нрш.леов( гйт г б пнтсиикор(о нпец
суе  випы м,гб лсн оо таке,е лсае досп е  ьыюмня 
аь
уо
ре аагвс ветре  де т реон иехия . 
то ерсб нроевлпгте.оюе лмее сасоестео ре  "тйяаоауые.азесзе пв моиа  чм геим ияти  оть вп дрт .сы с   ршжс тизнвь кон уи, ктко ьк ио и  ле,йоь-оя плисил,чгя твитн  аун неспаабпхсйд 

233941/233941 [==============================] - 37s 157us/sample - loss: 2.8099
Генерируем начиная с отрывка: "хоть  жизнь  наша  в  этом
  проявлении выходит за"
